In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.parameter import Parameter
import matplotlib.pyplot as plt
from timeit import default_timer
import sys
import os

from utilities3 import *
from networks import *

import warnings
warnings.filterwarnings("ignore")

torch.manual_seed(0)
np.random.seed(0)

In [2]:
expt = 'Expt-3'
save = True

In [3]:
if save==True:
    orig_stdout = sys.stdout
    q = open('Surrogate_'+str(expt)+'.txt', 'w')
    sys.stdout = q

### load data and data normalization

In [4]:
X_data = torch.tensor(np.load(str(expt)+'_inputs_cellcenters_nx1=395_nx2=395.npy'), dtype=torch.float)
Y_data = torch.tensor(np.load(str(expt)+'_outputs_cellcenters_nx1=395_nx2=395.npy'), dtype=torch.float)

print('-'*50)
print('shuffling the data:')
data_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(X_data, Y_data), batch_size=X_data.shape[0], shuffle=True)
for x, y in data_loader:
    X_data = x
    Y_data = y
print (X_data.shape, Y_data.shape)

print('-'*50)
print('splitting the data into train and test:')
ntrain = X_data.shape[0]-150
ntest = 150

X_train = X_data[0:ntrain]
Y_train = Y_data[0:ntrain]

print(X_train.shape, Y_train.shape)

X_test = X_data[-ntest:]
Y_test = Y_data[-ntest:]

print(X_test.shape, Y_test.shape)

print('-'*50)
print('normalizing the data:')
x_normalizer = GaussianNormalizer(X_train)
x_train = x_normalizer.encode(X_train)
x_test = x_normalizer.encode(X_test)

y_normalizer = GaussianNormalizer(Y_train)
y_train = y_normalizer.encode(Y_train)

print(x_train.shape, y_train.shape, x_test.shape)


In [5]:
#checks
print(x_train[0])
print(x_train[0].shape)

### training and testing

In [6]:
################################################################
# configs
################################################################
batch_size = 32
learning_rate = 0.001
epochs = 30

In [7]:
def sum_squared_error(a, b):
    return torch.sum((a-b)**2)

In [8]:
train_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(x_train, y_train), batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(x_test, Y_test), batch_size=batch_size, shuffle=False)

################################################################
# training and evaluation
################################################################
model = DenseResNet(dim_in=x_train.shape[1], dim_out=y_train.shape[1], num_resnet_blocks=5, 
                 num_layers_per_block=3, num_neurons=60, activation=nn.SiLU())
print(count_params(model))

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

for ep in range(epochs):
    
    model.train()
    t1 = default_timer()
    
    for x, y in train_loader:
        
        optimizer.zero_grad()
        output = model(x)
        target = y
        loss = sum_squared_error(output, target)/target.shape[0]
        loss.backward()

        optimizer.step()

    model.eval()
    with torch.no_grad():
        train_mse = sum_squared_error(y_normalizer.decode(model(x_train)), Y_train).item()/ntrain
        test_mse = 0.0
        for x, y in test_loader:
            output = y_normalizer.decode(model(x))
            test_mse += sum_squared_error(output, y).item()
        test_mse /= ntest 

    t2 = default_timer()
    print(ep, t2-t1, train_mse, test_mse)


In [9]:
print(y_normalizer.decode(model(x_test)))

In [10]:
print(Y_test)

In [11]:
if save==True:
    save_object(x_normalizer, 'x_normalizer_'+str(expt)+'.pkl')
    save_object(y_normalizer, 'y_normalizer_'+str(expt)+'.pkl')
    torch.save(model.state_dict(), 'model_state_dict_'+str(expt)+'.pt')

In [12]:
if save==True:
    sys.stdout = orig_stdout
    q.close()